In [1]:
using OrdinaryDiffEq, ForwardDiff, Distributions, RandomNumbers, NPZ, PyPlot, StaticArrays, HDF5, CSV, DelimitedFiles

In [ ]:
function initial_sampling(Eb,w1,mb)
    #nums=rand(1)*1.5*pi.-0.5*pi
    nums=rand(1)*2. *pi
    x1=cos(nums[1])*sqrt(Eb/(0.5*mb*w1*w1))
    y1=sin(nums[1])*sqrt(2*mb*Eb)
    return x1,y1
end
function bolz_energy(a)
    return rand(Truncated(Normal(1), 0., 5))
end
W=12.
dts=0.1*2. *pi/W
lista=[0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.,1.5]
lista=[0.3,0.4,0.5,0.6]
lista=[0.8,0.9,1.,1.5]
#lista=[0.3]
numeros=[1,2,3,5,7,10,13,15,20,25,30,35]
numeros=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#numeros=[4,12,14]
promedios=100
#lista=[0.3]
nnn=0.
for oo in lista
count=zeros(size(numeros)[1])

println(nnn/size(lista)[1])
nnn=nnn+1.
s=oo

probabilities=npzread("probs$s.npy")
vals=npzread("values$s.npy")
#fig, axes = subplots(nrows=2, ncols=1)
#subplot(2, 1, 1)
#plot(vals,probabilities)
#xlim([0.3,0.8])
d = Categorical(probabilities)



for ii in 1:size(numeros)[1]
#fig, axes = subplots(nrows=1, ncols=1)
ii=Int(ii)
println(numeros[ii])
N=numeros[ii]
Times=10000
w=zeros(N+1)
Es=zeros(N+1)
g=zeros(N+1)
E=zeros(N+1)
m=ones(N+1) .*0.1
g00=0.1
g=g .* 0. .+ g00/sqrt(N)
m[1]=1.
initial_position=zeros(N+1)
initial_momentum=zeros(N+1)        

        
@time for jj in 1:promedios
            

for mm in 1:N
    mm=Int(mm)
    w[mm+1] = vals[rand(d)]
end



Es=(bolz_energy.(Es))
Es=Es./(sum(Es[2:end]))
#initial_position=x0
#initial_momentum=p0

a=2.
b=1.

time=Times
#time=1.
tspan=(0., time)




function Hsys(q,p,N0)
    result=p[1]^2. /(2. *m[1]) - a*q[1]^2. /2. +b*q[1]^4. /4.
    for i in 1:N0
        result +=  -q[1]*g[i+1]*q[i+1]+p[i+1]^2. /(2. *m[2])+m[i+1]*w[i+1]^2. *q[i+1]^2. /2. + q[1]^2. *g[i+1]^2. /(2. *m[i+1]*w[i+1]^2.) 
    end
    return result
end
pdot(dp,p,q,params,t) = ForwardDiff.gradient!(dp, q->-Hsys(q, p,N), q)
qdot(dq,p,q,params,t) = ForwardDiff.gradient!(dq, p-> Hsys(q, p,N), p)
            
for i in 2:N+1
    i=Int(i)
    initial_position[i], initial_momentum[i]=initial_sampling(Es[i],w[i],m[i])
end            

prob = DynamicalODEProblem(pdot, qdot, initial_momentum, initial_position, tspan)
global sol = solve(prob, VerletLeapfrog(), dt=dts, abstol=1e-14, reltol=1e-14) #CalvoSanz4
pygui(false)
            

for bb in 1:size(sol[N+2,:])[1]
    bb=Int(bb)
    if abs(sol[N+2,bb])>1.
        for j in bb:size(sol[N+2,:])[1]-1
            if sol[N+2,j]<=0. && sol[N+2,j+1]>0.
                count[ii]=count[ii]+1
            elseif sol[N+2,j]>=0. && sol[N+2,j+1]<0.
                count[ii]=count[ii]+1
            end
        end
        break
    end
end

          

           


#plot(sol[N+2,:])            

end #promedios
show()
end #N
npzwrite("counts$s.npy",count./100.)
end #s
npzwrite("numeros.npy", numeros)


0.0
1


┌ Warning: `Truncated(d::UnivariateDistribution, l::Real, u::Real)` is deprecated, use `truncated(d, l, u)` instead.
│   caller = ip:0x0
└ @ Core :-1


263.220536 seconds (1.69 G allocations: 56.226 GiB, 29.84% gc time)
2
322.376338 seconds (2.64 G allocations: 102.349 GiB, 30.75% gc time)
3
377.795707 seconds (3.61 G allocations: 139.348 GiB, 27.92% gc time)
4
463.295633 seconds (4.58 G allocations: 215.906 GiB, 29.14% gc time)
5
531.699942 seconds (5.56 G allocations: 265.427 GiB, 28.07% gc time)
6
624.066407 seconds (6.53 G allocations: 370.160 GiB, 28.72% gc time)
7
678.092673 seconds (7.51 G allocations: 429.642 GiB, 28.82% gc time)
8
777.035667 seconds (8.48 G allocations: 560.842 GiB, 27.19% gc time)
9
828.239075 seconds (9.45 G allocations: 634.554 GiB, 27.36% gc time)
10
989.097266 seconds (10.43 G allocations: 794.782 GiB, 30.05% gc time)
11
1093.678169 seconds (11.40 G allocations: 877.601 GiB, 29.21% gc time)
12
1624.762611 seconds (24.16 G allocations: 1.233 TiB, 26.13% gc time)
13
1725.959811 seconds (26.11 G allocations: 1.326 TiB, 26.08% gc time)
14
1895.339516 seconds (28.06 G allocations: 1.434 TiB, 25.33% gc time)
1

In [30]:
nnn=[1,1,1,1,2,2,2,3,3,3,4,2,4,4,5,5,5]
xx=0
for bb in 1:size(nnn)[1]
    bb=Int(bb)
    if nnn[bb]>3
        xx=bb
        break
    end
end
nnn[xx+1]

2

In [32]:
abs(-10)

10